# Paper Implementation
Description: This is implementation of paper from M. Kachuee et al. "ECG Heartbeat Classification: A Deep Transferable Representation" [(source)](https://arxiv.org/abs/1805.00794)
* Dataset from kaggle: [(source)](https://www.kaggle.com/shayanfazeli/heartbeat)

### TODO next
- [x] implementation using dataset from kaggle (MITDB)
- [ ] implementation transfer learning for PTB (MI disease) dataset
- [ ] implementation using MITBIH dataset (selfmade implementation preprocessing method based on paper)

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="1";  

In [ ]:
from glob import glob
import tensorflow as tf
import pandas as pd
import numpy as np
#import re
import matplotlib.pyplot as plt
#import gc
import pickle
import pathlib
import io
from math import sqrt
nrs = 1895
np.random.seed(nrs)

from imblearn.over_sampling import SMOTE

from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,Callback,LearningRateScheduler
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Convolution1D, MaxPooling1D, GlobalMaxPool1D
from tensorflow.keras.layers import Conv1D, MaxPool1D,Add #cnn
from tensorflow.keras.layers import concatenate,Input
from tensorflow.keras import optimizers, losses, activations
from tensorflow.keras.models import Model
from tensorflow.keras.activations import relu

from tensorflow.keras import backend as K

print(tf.__version__)

#from sklearn.metrics import f1_score, accuracy_score
#from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import warnings
from datetime import datetime

from sklearn.metrics import classification_report
from sklearn.utils.multiclass import unique_labels
from collections import Counter

from calcCM import *

# print(gc.collect())

In [ ]:
## limit GPU memory usage
configproto = tf.compat.v1.ConfigProto()
configproto.gpu_options.per_process_gpu_memory_fraction = 0.3
configproto.gpu_options.allow_growth = True

#limit CPU logical core usage
configproto.intra_op_parallelism_threads=2
configproto.inter_op_parallelism_threads=2

#setup session config
sess = tf.compat.v1.Session(config=configproto)
tf.compat.v1.keras.backend.set_session(sess)

#### set up result folder

In [ ]:
# get current path directory
dirPath = os.path.abspath(os.getcwd())
print(dirPath)
#print(pathlib.Path().absolute())

In [ ]:
# create uniq folder name for results
now = datetime.now()

dt_string_daytime = now.strftime("%Y%m%d_%H%M%S")
dt_string_daytime += '_kachueeCNN-singleRun'
print("folder name =", dt_string_daytime)

In [ ]:
# create folder for results
data_result_path = list()
data_result_path.append('model_'+dt_string_daytime)
data_result_path.append(data_result_path[0]+'/ckpt')
data_result_path.append(data_result_path[0]+'/fitHistory')
data_result_path.append(data_result_path[0]+'/modelSummary')

for i in range(0,len(data_result_path)):
    if not os.path.exists(data_result_path[i]):
        print("create new folder "+data_result_path[i])
        os.makedirs(data_result_path[i])
    else:
        print(data_result_path[i]+" already created")

currentPath = dirPath+'/'+data_result_path[0]
print(currentPath)

In [ ]:
def activation_func(pos):
    if(pos== 1):
        return activations.relu
    elif(pos==2):
        return activations.sigmoid
    elif(pos==3):
        return activations.softmax
    elif(pos==4):
        return activations.tanh
    else:
        return activations.relu
    
def opt_func(pos,lr=0.0001):
    if(pos==1):
        return optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, amsgrad=False)
    elif(pos==2):
        return optimizers.Adagrad(learning_rate=lr,initial_accumulator_value=0.8)
    elif(pos==3):
        return optimizers.Nadam(learning_rate=lr, beta_1=0.9, beta_2=0.999)
    elif(pos==4):
        return optimizers.Adamax(learning_rate=lr, beta_1=0.9, beta_2=0.999)
    else:
        return optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, amsgrad=False)


In [ ]:
#Training parameter setup
nclass = 5
epochList = [1] # default 5
lr = 1e-3

modelID = 0
batchList = [100] # default 10


batchSize = batchList[0]
epochSize = epochList[0]

In [ ]:
#Deep CNN model architecture
global inp1

def conv_unit(unit, input_layer):
    s = '_' + str(unit)
    layer = Conv1D(name='Conv1' + s, filters=32, kernel_size=5, strides=1, padding='same', activation='relu')(input_layer)
    layer = Conv1D(name='Conv2' + s, filters=32, kernel_size=5, strides=1, padding='same', activation=None)(layer )
    layer = Add(name='ResidualSum' + s)([layer, input_layer])
    layer = Activation("relu", name='Act' + s)(layer)
    layer = MaxPooling1D(name='MaxPool' + s, pool_size=5, strides=2)(layer)
    return layer

def createModel(shape,lr = 0.001):

    ##=============
    inp1 = Input(shape=(shape,1),name='inp1')
    
    #model a1
    convLayer = Conv1D(32, kernel_size=5,name='conv1',strides=1)(inp1)
    
    for i in range(0,5):
        convLayer = conv_unit(i+1, convLayer)
    ##=============
    
    FC = Flatten()(convLayer)
    FC = Dense(32,activation='relu', name='FC1')(FC)
    FC = Dense(nclass,activation=activation_func(3), name='FC_out')(FC)

    model = Model(inputs=[inp1], outputs=FC)

    model.summary()
    model.compile(optimizer=opt_func(1,lr), 
                  loss='categorical_crossentropy',
                  metrics=["acc"])
    return model


In [ ]:
# dataset path
rootDataFolder = "kaggle dataset"

In [ ]:
# confusion matrix plot
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
# Training model
def train_model(xtr,ytr,xtx,ytx,savetype=0):
    tf.keras.backend.clear_session()
    num_batches_per_epoch = batchSize
    epoch = epochSize
    cvscores = []

    print("before SMOTE")
    counter = Counter(ytr)
    print(counter)
    print(ytr.shape)

    # apply synthetic oversampling
    oversample = SMOTE()
    trX, trY = oversample.fit_resample(xtr, ytr)
    
#     trX = np.copy(xtr)
#     trY = np.copy(ytr)
    print("after SMOTE")
    counter = Counter(trY)
    print(counter)
    print(trY.shape)
    
    trX = trX.reshape(trX.shape[0],trX.shape[1],1)
    trY = to_categorical(trY.astype(np.int),nclass)
    
    testX = xtx.reshape(xtx.shape[0],xtx.shape[1],1)
    testY = to_categorical(ytx.astype(np.int),nclass)
    
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                        initial_learning_rate=1e-3,
                        decay_steps=10000,
                        decay_rate=0.75)
    model = createModel(trX.shape[1],lr_schedule)
    
    file_path = currentPath+"/ckpt/"+"normal"
        
    if savetype == 0:
        ckpttype = ['min_loss','max_acc','last']

        fpl = list() #file path list
        for i in range(0, len(ckpttype)):
            fpl.append(file_path+"_"+ckpttype[i]+".h5")

        #print(fpl)

        callbacks_list = list()
        checkpoint_last = ModelCheckpoint(fpl[2], monitor='acc',period=epoch, verbose=1, save_best_only=False, mode='max')
        checkpoint_min_loss = ModelCheckpoint(fpl[0], monitor='loss', verbose=1, save_best_only=True, mode='min')
        checkpoint_max_acc = ModelCheckpoint(fpl[1], monitor='acc', verbose=1, save_best_only=True, mode='max')
        callbacks_list = [checkpoint_last,checkpoint_min_loss,checkpoint_max_acc]

    Wsave = model.get_weights()

    history = model.fit(trX, trY,batch_size=num_batches_per_epoch, epochs=epoch,
                        callbacks=callbacks_list, verbose=1,shuffle=True,
                        workers=1, use_multiprocessing=False)
#         history = model.fit(testX, testY,batch_size=num_batches_per_epoch, epochs=epoch,
#                             verbose=1,shuffle=True)

    #modelEvalTrain = model.evaluate(trX, trY,batch_size=num_batches_per_epoch, verbose=2)
    modelEvalTest = model.evaluate(testX, testY,batch_size=num_batches_per_epoch, verbose=2)

    #print("Model eval train result : {} - Model eval test result : {}".format(modelEvalTrain,modelEvalTest))
    print("Model eval test result : {}".format(modelEvalTest))
    print("%s: %.2f%%" % (model.metrics_names[1], modelEvalTest[1]*100))
    cvscores.append(modelEvalTest[1] * 100)

    model.set_weights(Wsave)
    

In [ ]:
# Training checkpoint evaluation
def evaluate_model(xtx,ytx):
    model = 0
    str_result = ""
    print(ckpt_root_dir)
    custom_msg  = model_root_dir[5:]

    ckpttype = ['min_loss','max_acc','last']
    #ckpttype = ['last']

    for i,ckpt_val in enumerate(ckpttype):
        cm = list()

        tf.keras.backend.clear_session()#clear session

        testX = xtx.reshape(xtx.shape[0],xtx.shape[1],1)
        testY = to_categorical(ytx.astype(np.int),nclass)

        if (model == 0):        
            model = createModel(testX.shape[1])
            strmodel = get_model_summary(model)
            summaryModelPath = currentPath+'/modelSummary/msum_'+model_root_dir+'.txt'
            save_modelSummary(strmodel,summaryModelPath)

        model.load_weights(ckpt_root_dir+'/'+'normal'+'_'+ckpt_val+'.h5')

        #predResult = model.predict_on_batch([testX])
        predResult = model.predict([testX])
        print("predshape {}".format(predResult.shape))

        ytrue = predictionToLabel(testY,nclass)
        ypred = predictionToLabel(predResult,nclass)
#         print(classification_report(ytrue, ypred, target_names=['1','2','3','4','5']))
#         print()
        
        disp = plot_confusion_matrix(y_true = ytrue,y_pred= ypred,
                                     #classes=np.array([0,1,2,3,4]),
                                     classes=np.array(['N','S','V','F','Q']),
                                 cmap=plt.cm.Blues,
                                 normalize=True)
        plt.show()
        print(confusion_matrix(ytrue,ypred,np.arange(nclass)))

In [ ]:
## personal note
pnote = "Deep CNN MITBIH training"
pnote = " \\n"
pnote = pnote+""

pathNote = data_result_path[0]+"\PNOTE.txt"

print(pnote)
with open(pathNote,'a') as f:
        f.write(pnote)

In [ ]:
# laod data
xtrain = np.load(rootDataFolder+"/x_train.npy")
ytrain = np.load(rootDataFolder+"/y_train.npy")
xtest = np.load(rootDataFolder+"/x_test.npy")
ytest = np.load(rootDataFolder+"/y_test.npy")

print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

In [ ]:
train_model(xtrain,ytrain,xtest,ytest)

In [ ]:
evaluate_model(xtest,ytest)

In [ ]:
# reset notebook after trainign to release resource (memory - RAM and GPU)
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")